# Custom Portfolio Comparison

This notebook demonstrates:
1. Creating custom portfolios with different asset allocations
2. Running multiple backtests
3. Comparing performance across strategies
4. Using RAS to determine which strategies are statistically superior

## 1. Load Data

In [ ]:
import pandas as pd
import rademacher_backtest as rbt

# Load price data
prices = pd.read_csv('data/sample_prices.csv', index_col='date', parse_dates=True)
loader = rbt.DataFrameLoader(prices)

print(f"Available assets: {', '.join(prices.columns)}")
print(f"Date range: {prices.index[0].date()} to {prices.index[-1].date()}")

## 2. Define Multiple Portfolio Strategies

Let's compare several classic portfolio allocations:

In [ ]:
# Define portfolio strategies
strategies = {
    'Conservative (80/20)': {
        'AGG': 0.80,  # 80% bonds
        'SPY': 0.20,  # 20% stocks
    },
    'Balanced (60/40)': {
        'SPY': 0.60,  # 60% stocks
        'AGG': 0.40,  # 40% bonds
    },
    'Aggressive (80/20)': {
        'SPY': 0.80,  # 80% stocks
        'AGG': 0.20,  # 20% bonds
    },
    'All Weather': {
        'SPY': 0.30,  # 30% stocks
        'AGG': 0.40,  # 40% bonds
        'GLD': 0.15,  # 15% gold
        'VNQ': 0.15,  # 15% real estate
    },
    'Global Diversified': {
        'SPY': 0.35,  # 35% US stocks
        'EEM': 0.15,  # 15% emerging markets
        'AGG': 0.30,  # 30% bonds
        'GLD': 0.10,  # 10% gold
        'VNQ': 0.10,  # 10% real estate
    },
}

print(f"Defined {len(strategies)} portfolio strategies")
for name in strategies:
    print(f"  - {name}")

## 3. Run All Backtests

Run each strategy through the backtest engine:

In [ ]:
# Common backtest parameters
backtest_params = {
    'loader': loader,
    'start_date': '2015-01-01',
    'end_date': '2023-12-31',
    'initial_capital': 100_000.0,
    'transaction_cost_bps': 10.0,
}

# Run backtests
results = {}
for name, portfolio in strategies.items():
    print(f"Running: {name}...")
    result = rbt.backtest(portfolio=portfolio, **backtest_params)
    results[name] = result

print(f"\n✅ Completed {len(results)} backtests")

## 4. Calculate Performance Metrics

Compare the performance of all strategies:

In [ ]:
# Calculate metrics for each strategy
perf_calc = rbt.PerformanceCalculator()
metrics_dict = {}

for name, result in results.items():
    metrics = perf_calc.calculate(result.daily_returns)
    metrics_dict[name] = metrics

# Create comparison DataFrame
comparison = pd.DataFrame({
    name: {
        'Final Value ($)': results[name].final_value,
        'Total Return (%)': results[name].total_return_pct,
        'CAGR (%)': metrics.cagr,
        'Volatility (%)': metrics.annualized_volatility,
        'Sharpe Ratio': metrics.sharpe_ratio,
        'Max Drawdown (%)': metrics.max_drawdown,
        'Calmar Ratio': metrics.calmar_ratio,
    }
    for name, metrics in metrics_dict.items()
}).T

# Display comparison
print("\n📊 Performance Comparison:\n")
print(comparison.round(2))

## 5. RAS Analysis for Multiple Strategies

Apply RAS methodology with multiple testing correction:

In [ ]:
# RAS analysis for each strategy
# Important: n_strategies accounts for multiple testing!
n_strategies = len(results)

ras_results = {}
for name, result in results.items():
    ras = rbt.analyze_ras(
        returns=result.daily_returns,
        confidence=0.99,
        n_strategies=n_strategies,  # Multiple testing correction
    )
    ras_results[name] = ras

# Create RAS comparison
ras_comparison = pd.DataFrame({
    name: {
        'Empirical Sharpe': ras.empirical_sharpe_annualized,
        'Adjusted Sharpe': ras.adjusted_sharpe_annualized,
        'Haircut': ras.total_haircut_annualized,
        'Stat. Positive': '✅' if ras.is_statistically_positive else '❌',
    }
    for name, ras in ras_results.items()
}).T

print("\n🔬 RAS Analysis (with multiple testing correction):\n")
print(ras_comparison)

## 6. Visualize Comparison

Plot cumulative returns for all strategies:

In [ ]:
import matplotlib.pyplot as plt

# Create cumulative returns for each strategy
plt.figure(figsize=(14, 7))

for name, result in results.items():
    cumulative = (1 + result.daily_returns).cumprod()
    plt.plot(cumulative.index, cumulative.values, label=name, linewidth=2)

plt.title('Cumulative Returns Comparison', fontsize=14, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Cumulative Return', fontsize=12)
plt.legend(loc='best')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 7. Risk-Return Scatter

Visualize the risk-return tradeoff:

In [ ]:
# Create risk-return scatter plot
plt.figure(figsize=(10, 7))

for name in strategies:
    vol = comparison.loc[name, 'Volatility (%)']
    ret = comparison.loc[name, 'CAGR (%)']
    stat_pos = ras_results[name].is_statistically_positive
    
    marker = 'o' if stat_pos else 'x'
    color = 'green' if stat_pos else 'red'
    
    plt.scatter(vol, ret, s=200, marker=marker, color=color, alpha=0.6)
    plt.annotate(name, (vol, ret), xytext=(5, 5), 
                textcoords='offset points', fontsize=9)

plt.xlabel('Volatility (Annual %)', fontsize=12)
plt.ylabel('CAGR (%)', fontsize=12)
plt.title('Risk-Return Profile\n(Green = Statistically Positive, Red = Not Significant)', 
         fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Summary

This example demonstrated:
1. ✅ Creating multiple portfolio strategies with different allocations
2. ✅ Running parallel backtests
3. ✅ Comparing performance metrics across strategies
4. ✅ Applying RAS with **multiple testing correction** (n_strategies parameter)
5. ✅ Visualizing risk-return tradeoffs

### Key Insights:

- The **multiple testing correction** in RAS is crucial when comparing strategies
- Higher Sharpe ratios don't always mean statistical significance
- Diversification can improve risk-adjusted returns
- The "best" empirical performer may not be statistically distinguishable from others

## Next Steps

- See `03_ras_analysis.ipynb` for deep dive into RAS methodology
- See `04_visualization.ipynb` for advanced charts